In [12]:
import numpy as np 
import cv2

In [13]:
camera = 1

In [14]:
# import intrinsic matrix and distortion coefficients
intrinsic_matrix = np.load(f'../intrinsic_matrices/{camera}.npy')
distortion_coefficients = np.load(f'../distortions/{camera}.npy')

print(intrinsic_matrix)
print(distortion_coefficients)

[[2.03425508e+03 0.00000000e+00 1.85906416e+03]
 [0.00000000e+00 2.05401046e+03 1.12199428e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[-0.30915611  0.09657182  0.0009885   0.00158603 -0.01378949]]


In [15]:
# import points
world_points = np.load(f'../extrinsic_calibration_points/{camera}_world.npy')
image_points = np.load(f'../extrinsic_calibration_points/{camera}_image.npy')
print(world_points)
print(image_points)


[[18000.  9000.     0.]
 [12000.  9000.     0.]
 [ 9000.  9000.     0.]
 [ 6000.  9000.     0.]
 [    0.  9000.     0.]
 [18000.     0.     0.]
 [12000.     0.     0.]
 [ 9000.     0.     0.]
 [ 6000.     0.     0.]]
[[1242. 1196.]
 [1042.  937.]
 [ 979.  852.]
 [ 929.  783.]
 [ 857.  682.]
 [2089. 1103.]
 [1761.  904.]
 [1638.  832.]
 [1532.  771.]]


In [16]:
# generate world points
# world_points = np.array([[0, 9, 0],
#                          [6, 9, 0],
#                          [12, 9, 0],
#                          [9, 9, 0],
#                          [0, 10.75, 0],
#                          [6, 10.75, 0]], dtype=np.float32) * 1000. # mm

# image_points = np.array([[2057, 1068],
#                          [1119, 698],
#                          [484, 446],
#                          [773, 560],
#                          [2351, 1009],
#                          [1413, 666]], dtype=np.float32) # px

# np.save(f'../extrinsic_calibration_points/{camera}_world', world_points)
# np.save(f'../extrinsic_calibration_points/{camera}_image', image_points)


In [17]:
# calibrate extrinsic matrix
success, rvecs, tvecs = cv2.solvePnP(world_points, image_points, intrinsic_matrix, distortion_coefficients)
print(success)

rotation_matrix, _ = cv2.Rodrigues(rvecs)

# convert from 3x3 to 4x4
rotation_matrix = np.pad(rotation_matrix, (0, 1), mode='constant')
rotation_matrix[3, 3] = 1

tvecs = tvecs.flatten() / 1000.

t_matrix = np.array([[1, 0, 0, tvecs[0]],
                     [0, 1, 0, tvecs[1]],
                     [0, 0, 1, tvecs[2]],
                     [0, 0, 0, 1]])

world_f_camera = np.dot(t_matrix, rotation_matrix)
extrinsic_matrix = np.linalg.inv(world_f_camera)
print(f'Extrinsic Matrix:\n{extrinsic_matrix}')

# save matrix 
np.save(f'../extrinsic_matrices/{camera}', extrinsic_matrix)

True
Extrinsic Matrix:
[[ 0.58707637  0.40299617 -0.70209361 31.28107091]
 [-0.78840822  0.08780006 -0.60885435 14.47564573]
 [-0.18372211  0.91098037  0.36927083 -7.02654039]
 [ 0.          0.          0.          1.        ]]
